# Text Preprocessing

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
"""Common Preprocessing Pipeline"""

# 1. clear text
# 2. drop stop words & to_lower()
# 3. tokenize (char, words, sentences)
# 4. stemming, lemmatize
# 5. encode, vectorize: (tfidf, CountVectorizer, HashingVectorizer)
# 6. Deploy

None

In [15]:
##############
# Import libs
##############

import numpy as np
import pandas as pd
import regex as re  # regex will be common in python soon
import html
import nltk
import pickle
from utils.dicts import apostrophe_dict, emoticon_dict, short_word_dict

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"  # handle multiple outputs

In [7]:
###########
# Settings
###########

STOP_WORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = nltk.stem.PorterStemmer()
LEMMATIZER = nltk.stem.wordnet.WordNetLemmatizer()

# Data load
train_df = pd.read_csv('data/train_tweets.csv')
test_df = pd.read_csv('data/test_tweets.csv')
df = train_df.append(test_df, ignore_index = True, sort = False)

In [8]:
df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


In [12]:
######################
# Preprocess Routine
######################
###################


def replace_from_dict(text, source_dict):
    
    """Search through text, map tokens via dict, place default if not in dict"""
    
    return " ".join([source_dict.get(word, word) for word in text.split()])


def remove_onechar_tokens(text):
    
    """Search through text, remove one-caracter tokens"""
    
    return ' '.join([w for w in text.split() if len(w)>1])


def filter_stop_words(tokens, stop_words=STOP_WORDS):
    
    """Remove stop words from tokens"""
    
    return [token for token in tokens if token not in stop_words]
    

def stem_tokens(tokens, stemmer=STEMMER):
    
    """Stemming preprocessing"""
    
    return [stemmer.stem(token) for token in tokens]


def lemmatize_tokens(tokens, lemmatizer=LEMMATIZER):
    
    """Lemmatize preprocessing"""
    
    return [lemmatizer.lemmatize(token) for token in tokens]
    

def preprocess(df,
               src_col='tweet',
               clean_col='clean_tweet',
               token_col='tweet_token',
               filter_col='tweet_token_filtered',
               stemmed_col='tweet_stemmed',
               lem_col='tweet_lemmatized'
              ):
     
        
    # 1. Clean html context
    df[clean_col] = df[src_col].apply(lambda x: html.unescape(x))
    
    # 2. Remove @user references
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'@[\w]*','', x))
    
    # 3. Correct register to lowercase
    df[clean_col] = df[clean_col].str.lower()
    
    # 4. Change apostrophes
    vfunc = np.vectorize(replace_from_dict)
    df[clean_col] = vfunc(df[clean_col], apostrophe_dict)
    
    # 5. Prolong short words
    df[clean_col] = vfunc(df[clean_col], short_word_dict)
    
    # 6. Replace emoticons
    df[clean_col] = vfunc(df[clean_col], emoticon_dict)
    
    # 7. Replace punctuation to spaces
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^\w\s]','', x))
    
    # 8. Replace special characters to spaces
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
    
    # 9. Replace nums for spaces
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
    
    # 10. Drop one char words
    vfunc = np.vectorize(remove_onechar_tokens)
    df[clean_col] = vfunc(df[clean_col])
    
    # 11. Tokenize text
    df[token_col] = df[clean_col].apply(lambda x:  nltk.tokenize.word_tokenize(x))
    
    # 12. Filter stop words
    stop_words = set(nltk.corpus.stopwords.words("english"))
    df[filter_col] = df[token_col].apply(lambda x: filter_stop_words(x))
    
    # 13. Apply stemming
    df[stemmed_col] = df[filter_col].apply(lambda x: stem_tokens(x))
    
    # 14. Lemmatize
    df[lem_col] = df[stemmed_col].apply(lambda x: lemmatize_tokens(x))
    
    return df
    

In [13]:
%%time

result = preprocess(df)

Wall time: 42.8 s


In [17]:
# 15. Save to_pickle
result.to_pickle("output/df_processed.pkl")